# Prelude

In [ ]:
# Enable interactive plots (%matplotlib -l to list backends)
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from bgcellmodels.common import analysis, signal, units, treeutils
from bgcellmodels.common.protocols import pick_random_segments, sample_tree_uniformly
from bgcellmodels.mechanisms import noise, synapses
import bgcellmodels.models.STN.GilliesWillshaw.gillies_model as gillies_model
# import bgcellmodels.models.STN.GilliesWillshaw.gillies_pynn_model as gillies_model

import neuron; h = neuron.h
import bluepyopt.ephys as ephys

# print code version (hash of checked out version)
print("\nCurrent commit:")
!git log -1
# print("\nChanges since last commit:")
# !git status --short

# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd ..

# Create cell

In [ ]:
soma, dends, stims = gillies_model.stn_cell_gillies()
dendritic = list(dends[0]) + list(dends[1])

In [ ]:
# Create cell using PyNN/Ephys wrapper
# cell = gillies_model.StnCellModel(membrane_noise_std=0.0)
# icell = cell.icell
# nrnsim = cell.sim

# named_seclists =  {listname: list(getattr(icell, listname)) for listname in cell.seclist_names}
# for k, v in named_seclists.items():
#     if len(v)==0:
#         named_seclists.pop(k) # don't include empty SectionLists
#     else:
#         print("{} : {} sections".format(k, len(v)))

# somatic = named_seclists['somatic']
# dendritic = named_seclists['basal']

# soma = somatic[0]
# dend = dendritic[0]

# nseg = sum((sec.nseg for sec in icell.all))
# print("Total number of compartments: {}".format(nseg))

## Synapse Locations

A suitable compartment can be selected either using `treeutils.subtree_topology` or by using 
NEURON GUI -> Tools -> Distributed Mechanisms > Viewers > Shape Name

In [ ]:
# inspect topology to determine good synapse location
# print(treeutils.subtree_topology(soma, max_depth=1))

# Pick segments on separate subtrees if you don't want simultaneous spikes to interfer
distal_seg = dendritic[8](0.75)
middle_seg = dendritic[20](0.25)
proximal_seg = dendritic[24](0.5)

In [ ]:
plt.figure()
ax = plt.subplot(111, projection='3d')
ax.view_init(90,90) # set azimut and elevation
ax.set_zlim((-1,1)) # prevent rounding errors
ax.set_axis_off() # hide x,y,z axes and grid

# Plot shape of our cell
h.define_shape(sec=soma) # for cells without 3D morphology
morphology.shapeplot(h, ax)

# Plot soma locations
markers = ('og', 'ob', 'or')
labels = ['prox', 'mid', 'dist']
for i, seg in enumerate((proximal_seg, middle_seg, distal_seg)):
    print(labels[i] + " : " + seg.sec.name())
    morphology.mark_locations(h, seg.sec, seg.x, label=labels[i], markspec=markers[i])

plt.legend()
plt.show()

# Membrane Currents

Magnitude of membrane currents as a reference for magnitude of synaptic currents

## Experimental Data

### Article Do & Bean (2003)

Article "Subthreshold Sodium Currents and Pacemaking of Subthalamic Neurons: Modulation by Slow Inactivation".

Fig. 2B shows that peak TTX-sensitive Na current during action potential is approx 600 pA.
- `max(I_NaF) = 600 pA`

### Article Bevan (2007)

Article "GABAergic control of the subthalamic nucleus".

Fig. 7B confirms that peak TTX-sensitive Na current is ~ 0.6 nA = 600 pA
- `max(I_NaF) = 600 pA`

### Article Gillies & Willshaw (2006)

Peak somatic membrane current (total, not only Na but should be same) is `-5 nA = -5000 pA` which is order of magnitude larger than in vitro. This could have many reasons:
- the membrane capacitance or area are not well calibrated.
- the voltage clamp setup used in papers does not record the same fraction of current as recorded in simulation
    - current flowing away to dendrites?
    - patch of membrane does not have same area adn therefore total current?

### Relation Synaptic Currents

In STN synaptic currents are reported in order of several hundred pA (350 pA for GABA currents).

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'I_soma': {'var':'i_membrane_', 'sec':'soma', 'loc':0.5},
}

# Compute absolute membrane current (nA) in var 'i_membrane_'
cvode = h.cvode # Created by Gillies model
cvode.use_fast_imem(True) # Compute i_membrane in each segment
cvode.cache_efficient(True) # necessary for lfp, also 33% reduction in simulation time

# Record
rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.set_aCSF(4) # Hoc function defined in Gillies code
h.v_init = -68.0
h.tstop = 2000.0
h.run()

In [ ]:
# Plot recorded traces
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt

# Plot synaptic traces
fig, axes = plt.subplots(2,1)

ax = axes[0]
ax.plot(t_soma, v_soma, 'b', label='V_soma')
ax.set_ylabel('voltage (mV)')
ax.grid(True)
ax.legend()

ax = axes[1]
ax.plot(t_soma, vec_dict['I_soma'].as_numpy(), 'r', label='I_soma')
ax.set_ylabel('current (nA)')
ax.legend()

# CTX Afferents

## Experimental Data

The paper by <span style='color:blue;font-weight:bold'> Chu (2015) </span> :

For excitatory synapses (CTX -> STN), the paper by Chu (2015) about GP physiology shows AMPA and NMDA synapse properties in Figure 1.

**For AMPA**: Fig. 1E bottom left trace shows time course of an EPSC evoked when the cell is near its resting membrane potential:
- EPSC amplitude is ~ `275 pA`
    + 275e-3 nA / (-80 mV - 0 mV) = -275e-3/80 uS = 3.44 nS
- time constant for rising phase (exponential) : ~ 1.0 ms
- time constant for decay phase (exponential) : ~ 4.0 ms


**For NMDA**: Fig. 1E top right trace shows time course of an EPSP evoked when the cell is at 40 mV which is close to the 30 mV at which Mg2+ block is removed from NMDAR.
- EPSC amplitude is ~ `210 pA`
    + 210e-3 nA / (30 mV - 0 mV) = 210e-3 / 30 uS = 210/30 nS = 7 nS
- time constant for rising phase (exponential) : ~ 3.7 ms
- time constant for decay phase (exponential) : ~ 80.0 ms

**STD**: Short term depression:

- <span style='color:blue;font-weight:bold'>Steiner (2019)</span>  Fig.6 and text Results - Sec. 4 gives STD ratios for GLU and GABA afferents to STN
         - STD ratios (5th/1st synaptic amplitude) are given for GLU and GABA afferets at 10; 20; 130 Hz:
         - EPSC (GLU): 0.44 ; 0.32 ; 0.07
         - IPSC (GABA): 0.58 ; 0.58 ; 0.46 
         
- <span style='color:blue;font-weight:bold'> Froux (2017) </span> (Fig. 3) shows STD data

The paper by <span style='color:blue;font-weight:bold'> Chu (2017) </span> (Fig. S1) shows AMPA:NMDA ratio (in DD condition, unchanged from control).



## Calibration Results
### Results Summary

AMPA:
- Single synapse (no multi-synapse rule) with reported conductance of 3.44 nS yields plausible EPSP.

- time constant of 200 ms and base release probability of 0.7 fits data using Tsodyks-Markram model.
    - *TODO*: find out which data, notes say Gradinaru 2009 but not found

### CTX -> STN (GLU)

In [ ]:
# Calibrate AMPA synapse to get desired PSP time course

# Save inputs
stim_data = {}
istim = None

def make_ctx_afferents(segments, rate=50, noise=0, num_spikes=1e9, tstart=750, 
                       gsyn_base=14e-3, scale_gsyn=1.0,
                       intraburst_rate=0, interburst_interval=200, 
                       burst_duration=50.0, burst_noise=1.0):
    # Clear previous inputs
    stim_data['CTX'] = {}
    stim_data['CTX']['synapses'] = []
    stim_data['CTX']['netstims'] = []
    stim_data['CTX']['netcons'] = []
    
    if isinstance(tstart, (float, int)):
        tstart = [tstart]*len(segments)
    
    # Make new inputs
    for i, seg in enumerate(segments):
        syn = h.GLUsyn(seg)
        stim_data['CTX']['synapses'].append(syn)
        
        # One spike generator for background spikes
        stim = h.NetStim()
        stim.interval = rate**-1*1e3
        stim.number = num_spikes
        stim.noise = noise
        stim.start = tstart[i]
        
        nc = h.NetCon(stim, syn)
        nc.delay = 1.0
        nc.weight[0] = 1.0
        
        stim_data['CTX']['netstims'].append(stim)
        stim_data['CTX']['netcons'].append(nc)
        
        # One spike generator for bursts (does not spike between bursts)
        if intraburst_rate != 0:
            bstim = h.BurstStim()
            bstim.fast_invl = intraburst_rate ** -1 * 1e3
            bstim.slow_invl = interburst_interval
            bstim.burst_len = burst_duration * 1e-3 * intraburst_rate
            bstim.start = tstart[i]
            bstim.noise = burst_noise
            
            bnc = h.NetCon(bstim, syn)
            bnc.delay = 1.0
            bnc.weight[0] = 1
            
            stim_data['CTX']['netstims'].append(bstim)
            stim_data['CTX']['netcons'].append(bnc)
        
        # Common STP parameters
        syn.tau_rec = 200.0
        syn.tau_facil = 1.0
        syn.U1 = P_release = 0.4
        
        # AMPA synapse parameters
        syn.gmax_AMPA = gAMPA = gsyn_base * scale_gsyn
        syn.tau_r_AMPA = 1.0
        syn.tau_d_AMPA = 4.0

        # NMDA synapse parameters
        syn.gmax_NMDA = gAMPA
        syn.tau_r_NMDA = 3.7
        syn.tau_d_NMDA = 80

In [ ]:
# Clear existing inputs
stim_data = {}
istim = None
hpwa = None
fih = None

h.distance(0, 0.5, sec=soma)
can_have_synapse = lambda seg: h.distance(1, seg.x, sec=seg.sec) >= 120.0

# Multi-synapse calibration
# make_ctx_afferents(pick_random_segments(dendritic, 8, can_have_synapse),
#                    gsyn_base=2.0e-2, scale_gsyn=1.0,
#                    rate=60, noise=0, num_spikes=100,
#                    tstart=750)

# Single synapse calibration
make_ctx_afferents((proximal_seg, middle_seg, distal_seg),
                   gsyn_base=14e-3, scale_gsyn=1.0,
                   rate=100.0, noise=0, num_spikes=100,
                   tstart=[750+i*600 for i in range(3)])

# Disable spiking for more accurate PSP measurement
# for sec in h.allsec():
#     sec.gmax_NaF = 0.0

# Alternative to disable spiking: hyperpolarizing current (like in article figure)
istim = h.IClamp(soma(0.5))
istim.delay = 250
istim.dur = 1000
istim.amp = -0.2 # 100 pA = 0.1 nA

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'prox': proximal_seg,
    'mid': middle_seg,
    'dist': distal_seg,
    'synGLU': stim_data['CTX']['synapses'][0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'V_prox': {'var':'v', 'seg':'prox'},
    'V_mid': {'var':'v', 'seg':'mid'},
    'V_dist': {'var':'v', 'seg':'dist'},
    # Synapse variables
    'gAMPA': {'pointp':'synGLU', 'var':'g_AMPA'},
    'gNMDA': {'pointp':'synGLU', 'var':'g_NMDA'},
    'iAMPA': {'pointp':'synGLU', 'var':'i_AMPA'},
    'iNMDA': {'pointp':'synGLU', 'var':'i_NMDA'},
    'iGLU': {'pointp':'synGLU', 'var':'i'},
    'Rrp': {'pointp':'synGLU', 'var':'R'},
    'Use': {'pointp':'synGLU', 'var':'Use'},
}

# Record
rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.set_aCSF(4) # Hoc function defined in Gillies code
h.v_init = -68.0
h.tstop = 2000.0
# h.init()
h.run()

In [ ]:
# Plot recorded traces
plot_interval = (700,1200) # (700, 1000)

plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
# plt.ylim((-90, -70))
# plt.xlim(plot_interval)
plt.grid(True)

# Plot synaptic traces
fig, axes = plt.subplots(2,1)

ax = axes[0]
ax.plot(t_soma, vec_dict['gAMPA'].as_numpy(), 'b', label='gAMPA')
ax.plot(t_soma, vec_dict['gNMDA'].as_numpy(), 'r', label='gNMDA')
ax.set_ylabel('conductance (uS)')
ax.set_xlim(plot_interval)
ax.legend()

ax = axes[1]
ax.plot(t_soma, vec_dict['iAMPA'].as_numpy(), 'b', label='iAMPA')
ax.plot(t_soma, vec_dict['iNMDA'].as_numpy(), 'r', label='iNMDA')
ax.plot(t_soma, vec_dict['iGLU'].as_numpy(), 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.set_xlim(plot_interval)
ax.legend()

# figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True, timeRange=plot_interval) # yRange=(-80,40), timeRange=

### AMPA Only

In [ ]:
# Calibrate AMPA synapse to get desired PSP time course

spike_interval = 150.0
synapses, stims, netcons = [], [], []

conductance_nS = 3.44
tau_rise = 1.0
tau_decay = 4.0
tau_rec = 200.0
tau_facil = 1.0
P_release = 0.7

for i, seg in enumerate((proximal_seg, middle_seg, distal_seg)):
    syn = h.GLUsyn(seg)
    
    stim = h.NetStim()
    stim.number = 1
    stim.noise = 0
    stim.start = 750 + i*spike_interval
    
    nc = h.NetCon(stim, syn)
    nc.delay = 1.0
    
    # Synapse parameters
    #nc.weight[0] = conductance_nS / P_release # conductance in [nS]
    nc.weight[0] = 1.0
    syn.gmax_AMPA = conductance_nS / P_release * 1e-3 # [uS] or set netcon weight in [nS]
    syn.gmax_NMDA = 0.0
    syn.tau_r_AMPA = tau_rise
    syn.tau_d_AMPA = tau_decay
    
    syn.tau_rec = tau_rec
    syn.tau_facil = 1.0
    syn.U1 = P_release # release probability
    
    # Save refs
    synapses.append(syn)
    stims.append(stim)
    netcons.append(nc)

# Disable spiking for more accurate PSP measurement
# for sec in h.allsec():
#     sec.gmax_NaF = 0.0

# Alternative to disable spiking: hyperpolarizing current (like in article figure)
istim = h.IClamp(soma(0.5))
istim.delay = 250
istim.dur = 1000
istim.amp = -0.2 # 100 pA = 0.1 nA

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'prox': proximal_seg,
    'mid': middle_seg,
    'dist': distal_seg,
    'synGLU': synapses[0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'V_prox': {'var':'v', 'seg':'prox'},
    'V_mid': {'var':'v', 'seg':'mid'},
    'V_dist': {'var':'v', 'seg':'dist'},
    # Synapse variables
    'gAMPA': {'pointp':'synGLU', 'var':'g_AMPA'},
    'gNMDA': {'pointp':'synGLU', 'var':'g_NMDA'},
    'iAMPA': {'pointp':'synGLU', 'var':'i_AMPA'},
    'iNMDA': {'pointp':'synGLU', 'var':'i_NMDA'},
    'iGLU': {'pointp':'synGLU', 'var':'i'},
    'Rrp': {'pointp':'synGLU', 'var':'R'},
    'Use': {'pointp':'synGLU', 'var':'Use'},
}

# Record
rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.set_aCSF(4) # Hoc function defined in Gillies code
h.v_init = -68.0
h.tstop = 2000.0
h.init()
# h.run()
nrnsim.run(h.tstop, h.dt)

In [ ]:
# Plot recorded traces
plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.grid(True)

# Plot synaptic traces
fig, axes = plt.subplots(2,1)

ax = axes[0]
ax.plot(t_soma, vec_dict['gAMPA'].as_numpy(), 'b', label='gAMPA')
ax.plot(t_soma, vec_dict['gNMDA'].as_numpy(), 'r', label='gNMDA')
ax.set_ylabel('conductance (uS)')
ax.legend()

ax = axes[1]
ax.plot(t_soma, vec_dict['iAMPA'].as_numpy(), 'b', label='iAMPA')
ax.plot(t_soma, vec_dict['iNMDA'].as_numpy(), 'r', label='iNMDA')
ax.plot(t_soma, vec_dict['iGLU'].as_numpy(), 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.legend()

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

### NMDA Only

In [ ]:
# Calibrate NMDA synapse to get desired PSP time course
# NOTE: it is important to deliver spikes when Mg2+ gate is removed
#       e.g. by timing a pSP to arrive during a backpropagating spike
#       or after AMPA spikes

spike_interval = 150.0
synapses, stims, netcons = [], [], []

conductance_nS = 7.0
tau_rise = 3.7
tau_decay = 80.0
tau_rec = 200.0
tau_facil = 1.0
P_release = 0.7

for i, seg in enumerate((proximal_seg, middle_seg, distal_seg)):
    syn = h.GLUsyn(seg)
    
    stim = h.NetStim()
    stim.number = 1
    stim.noise = 0
    stim.start = 750 + i*spike_interval
    
    nc = h.NetCon(stim, syn)
    nc.delay = 1.0
    
    # Synapse parameters
    nc.weight[0] = conductance_nS / P_release # conductance in [nS]
    # syn.gmax_AMPA = 0.01 # [uS] or set netcon weight in [nS]
    syn.gmax_AMPA = 0.0
    syn.tau_r_NMDA = tau_rise
    syn.tau_d_NMDA = tau_decay
    
    syn.tau_rec = tau_rec
    syn.tau_facil = 1.0
    syn.U1 = P_release # release probability
    
    # Save refs
    synapses.append(syn)
    stims.append(stim)
    netcons.append(nc)

# Disable spiking for more accurate PSP measurement
# for sec in h.allsec():
#     sec.gmax_NaF = 0.0

# Alternative to disable spiking: hyperpolarizing current (like in article figure)
istim = h.IClamp(soma(0.5))
istim.delay = 250
istim.dur = 1000
istim.amp = -0.2 # 100 pA = 0.1 nA

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'prox': proximal_seg,
    'mid': middle_seg,
    'dist': distal_seg,
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'V_prox': {'var':'v', 'seg':'prox'},
    'V_mid': {'var':'v', 'seg':'mid'},
    'V_dist': {'var':'v', 'seg':'dist'},
}

# Record
rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.set_aCSF(4) # Hoc function defined in Gillies code
h.v_init = -68.0
h.tstop = 2000.0
h.init()
# h.run()
nrnsim.run(h.tstop, h.dt)

# Plot recorded traces
plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.grid(True)

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

# GPE Afferents

## Experimental Data

### GABA-A Data

For **GABA-A synapses** (GPe -> STN):

The paper by <span style='color:blue;font-weight:bold'> Chu (2015) </span> about GP physiology shows GABA-A ISPSc examples in Figures 2,3,4,5,6.

- IPSC amplitude is ~ `350 pA`
    + 350e-3 nA / (-85 mV - -68 mV) = -350e-3/17 uS = 20.6 nS
- time constant for rising phase (exponential) : ~ 2.6 ms
- time constant for decay phase (exponential) : ~ 5.0 ms

The paper by <span style='color:blue;font-weight:bold'> Fan (2012) </span> shows GABA-A ISPSs and distribution of conductances in Fig. 2, and distributions of decay times in Fig. 1 (mISPC decay times should be same as for spike-induced IPSC).

- reported conductance is ~7 nS
    - median ~= 7 nS
    - sigma ~= 3 nS
- time constant for rising phase (exponential) : ~ 1.0 ms
- time constant for decay phase (exponential) : ~ 6.0 ms

The paper by <span style='color:blue;font-weight:bold'> Atherton (2013) </span> shows GABA-A IPSCs and short-term depression (STD) triggered by successive high-frequency IPSCs.

- exponential time constant of recovery is reported as 17.3 +/- 18.9 seconds
    - see Fig 2B and legend


- Fig 12 & 14 show magnitude of IPSPs for approx. 60 presynaptic inhibitory neurons foring synchronously
    - this indicates that indeed unitary conductances should be around 7 nS


**STD**: time scale of depression:

- <span style='color:blue;font-weight:bold'>Atherton (2013)</span>:
    - Fig. 2.D, 3.D : stready state magnitude @ 33 Hz is 15%, @100 Hz is 5%
    - Fig. 1.A, 3.B, 6.A, 7.A, 8.A, 9.A (examples of synaptic traces)
        - takes 300-500 ms with slow drop-off to about 30-50% magnitude, and then suddenly it's fully depleted
    - Fig 2C-D: the x-axis means that the conductance distribution (min, mean, max) is for IPSCs falling between [i-1, i] seconds. So in the first second of 33 Hz stimulation the average amplitude is ~ 30% of peak amplitude, and in the second second it is ~ 12%.
    

- <span style='color:blue;font-weight:bold'>Steiner (2019)</span> Fig.6 and text Results - Sec. 4 gives STD ratios for GLU and GABA afferents to STN
         - STD ratios (5th/1st synaptic amplitude) are given for GLU and GABA afferets at 10; 20; 130 Hz:
         - EPSC (GLU): 0.44 ; 0.32 ; 0.07
         - IPSC (GABA): 0.58 ; 0.58 ; 0.46 


### GABA-B Data

For **GABA-B synapses** (GPe -> STN):

For inhibitory synapses (STR), the paper by <span style='color:blue;font-weight:bold'>Levine, Hull, Buchendwald (1974)</span> about GP physiology shows following responses in Figure 1 (using electrical stimulation, likely stimulating large fraction of afferents):

- average IPSP trace shows approx. 5 mV dip from baseline
- exponential time constant for rising (dipping) phase : 66.0 ms
- exponential time constant for recovery phase : 100.0 ms

## Calibration Results

By cross-referencing and comparing data from three articles mentioned above, the conclusion is that the unitary conductance (single synapse) should approx. 7 nS as reported in Fan (2012). Hence for GABA-A we get:

- gsyn = 7 nS
- tau_rec = 17300.0
- tau_facil = 1.0
- P_release = 0.2


To calibrate a single GABAB synapse positioned at a proximal/trunk section, receiving volley of 8 spikes at 100 Hz to yield the IPSP shown in Figure 1, the following parameters work:

- gsyn = 0.0375 nS
- tau_rise_NT = 5.0 ms
- tau_decay_NT = 10.0 ms


**Calibration using `GABAsynTM2`:**

Following settings yield IPSP down to -80 mV when 8 synapses co-activated

```python
syn.tau_rec_A = 500.0
syn.tau_facil_A = 1.0
syn.U1_A = 0.2
syn.gmax_GABAA = 2.0e-2
syn.tau_r_GABAA = 2.0
syn.tau_d_GABAA = 5.0

syn.tau_rec_B = 400.0
syn.tau_facil_B = 200.0
syn.U1_B = 0.15
syn.gmax_GABAB = 0.25 * gGABAA
syn.tau_r_GABAB = 5.0
syn.tau_d_GABAB = 50.0
```

### GPE -> STN (GABA)

In [ ]:
# Save inputs
stim_data = {}
istim = None

def make_gpe_afferents(segments, rate=50, noise=0, num_spikes=1e9, tstart=750, 
                       gsyn_base=1e-4, scale_gsyn=1.0, 
                       intraburst_rate=0, interburst_interval=200, 
                       burst_duration=50.0, burst_noise=1.0, cascade=False,
                       **kwargs):
    """
    @param    **kwargs : parameters applied to each synapse
    """
    # Clear previous inputs
    stim_data['GPE'] = {}
    stim_data['GPE']['synapses'] = []
    stim_data['GPE']['netstims'] = []
    stim_data['GPE']['netcons'] = []
    
    if isinstance(tstart, (float, int)):
        tstart = [tstart]*len(segments)
    
    # Make new inputs
    last_segment, last_synapse = None, None
    for i, seg in enumerate(segments):
        if seg is last_segment:
            syn = last_synapse
        elif cascade:
            syn = h.GABAsyn2(seg)
        else:
            syn = h.GABAsynTM2(seg)
        stim_data['GPE']['synapses'].append(syn)
        
        # One spike generator for background spikes
        stim = h.NetStim()
        stim.interval = rate**-1*1e3
        stim.number = num_spikes
        stim.noise = noise
        stim.start = tstart[i]
        
        nc = h.NetCon(stim, syn)
        nc.delay = 1.0
        nc.weight[0] = 1.0
        
        stim_data['GPE']['netstims'].append(stim)
        stim_data['GPE']['netcons'].append(nc)
        
        # One spike generator for bursts (does not spike between bursts)
        if intraburst_rate != 0:
            bstim = h.BurstStim()
            bstim.fast_invl = intraburst_rate ** -1 * 1e3
            bstim.slow_invl = interburst_interval
            bstim.burst_len = burst_duration * 1e-3 * intraburst_rate
            bstim.start = tstart[i]
            bstim.noise = burst_noise
            
            bnc = h.NetCon(bstim, syn)
            bnc.delay = 1.0
            bnc.weight[0] = 1
            
            stim_data['GPE']['netstims'].append(bstim)
            stim_data['GPE']['netcons'].append(bnc)
        
        # WARNING: set synapse parameters using kwargs !!!
        if cascade:
            # See notes for configuring synapse in GABAsynTmGprot.mod
            syn.U1 = 0.2 # Atherton: 0.1 / LuNetStnGpe: 0.2
            syn.tau_rec = 400.0 # Atherton: 1250.0 / LuNetStnGpe: 400
            syn.tau_facil = 1.0 # Atherton: 50.0 / LuNetStnGpe: 1
            
            syn.gmax_GABAA = gGABAA = gsyn_base * scale_gsyn 
            syn.tau_r_GABAA = 2.0
            syn.tau_d_GABAA = 7.0
            
            syn.gmax_GABAB = 0.2 * gGABAA
            syn.tau_r_GABAB = 5.0
            syn.tau_d_GABAB = 25.0
            syn.KD = 1.4 # Atherton: 0.75
            syn.n = 4.0
            syn.K3 = 0.098
            syn.K4 = 0.0065 # LuNetDBS: 0.00625 / 0.0075 -> steady state is 1/2 of that using 0.00625
        else:
            # GABA-A synapse parameters
            syn.tau_rec_A = 1250.0
            syn.tau_facil_A = 50.0
            syn.U1_A = P_release = 0.1 # release probability
            syn.gmax_GABAA = gGABAA = gsyn_base * scale_gsyn # gmax is in [uS]
            syn.tau_r_GABAA = 2.0
            syn.tau_d_GABAA = 7.0

            # GABA-B synapse parameters
            syn.tau_rec_B = 400.0
            syn.tau_facil_B = 200.0
            syn.U1_B = 0.15 # release probability
            syn.gmax_GABAB = 0.35 * gGABAA
            syn.tau_r_GABAB = 5.0 # initial species of signaling cascade
            syn.tau_d_GABAB = 100.0 # initial species of signaling cascade
            
        for k, v in kwargs.items():
            setattr(syn, k, v)
        
        last_segment = seg
        last_synapse = syn

In [ ]:
# Clear existing inputs
stim_data = {}
istim = None
hpwa = None

h.distance(0, 0.5, sec=soma)
can_have_synapse = lambda seg: h.distance(1, seg.x, sec=seg.sec) < 120.0

# Multi-synapse calibration
# make_gpe_afferents(pick_random_segments(dendritic, 8, can_have_synapse),
#                    gsyn_base=0.5e-2, scale_gsyn=1.0,
#                    rate=60, noise=0, num_spikes=10,
#                    tstart=750)

# Single synapse calibration
destexhe_cascade = True
make_gpe_afferents((proximal_seg,), # , proximal_seg, proximal_seg),
                   gsyn_base=2.3e-2, scale_gsyn=1.0,
                   # Stimulation pattern
                   rate=130.0, noise=0, num_spikes=100,
                   tstart=[750+i*300 for i in range(3)],
                   cascade=destexhe_cascade,
                   # Synapse parameters
                   U1=0.2,
                   tau_rec=400.0,
                   tau_facil=1.0,
                   # K4=0.0065
                  )

# Disable spiking for more accurate PSP measurement
# for sec in h.allsec():
#     sec.gmax_NaF = 0.0

# Alternative to disable spiking: hyperpolarizing current (like in article figure)
surf_factor = 0.01

istim = h.IClamp(soma(0.5))
istim.delay = 250
istim.dur = 1000
istim.amp = 0.0 * surf_factor # 100 pA = 0.1 nA

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'prox': proximal_seg,
    'mid': middle_seg,
    'dist': distal_seg,
    'synGABA': stim_data['GPE']['synapses'][0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
#     'V_prox': {'var':'v', 'sec':'prox', 'loc':0.5},
#     'V_mid': {'var':'v', 'sec':'mid', 'loc':0.5},
#     'V_dist': {'var':'v', 'sec':'dist', 'loc':0.5},
    'gGABAA': {'pointp':'synGABA', 'var':'g_GABAA'},
    'gGABAB': {'pointp':'synGABA', 'var':'g_GABAB'},
    'Rrp': {'pointp':'synGABA', 'var':'Rrp_B'},
    'Use': {'pointp':'synGABA', 'var':'Use_B'},
    'Itot': {'pointp':'synGABA', 'var':'i'},
    'gtot' : {'pointp':'synGABA', 'var':'g'},
}

if destexhe_cascade:
    trace_specs['Rrp'] = {'pointp':'synGABA', 'var':'Rrp'}
    trace_specs['Use'] = {'pointp':'synGABA', 'var':'Use'}
    trace_specs['G'] = {'pointp':'synGABA', 'var':'G'}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.set_aCSF(4) # Hoc function defined in Gillies code
h.v_init = -68.0
h.tstop = 3000.0
#h.init()
h.run()

In [ ]:
# Plot recorded traces
plt.figure()
vrec_soma = vec_dict['V_soma']
v_soma = vrec_soma.as_numpy()
t_soma = t_rec = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.grid(True)

# Plot GABAsyn variables automatically
#figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

# Plot GABAsyn variables manually
fig, axes = plt.subplots(3, 1, figsize=(10,6))
interval = (740, 1500.0) # (740, h.tstop)
a = int(interval[0] // rec_dt)
b = int(interval[1] // rec_dt)
t_plot = t_rec[a:b]

ax = axes[0]
ax.plot(t_plot, vec_dict['gGABAA'].as_numpy()[a:b], 'b', label='gGABAA')
ax.plot(t_plot, vec_dict['gGABAB'].as_numpy()[a:b], 'r', label='gGABAB')
ax.plot(t_plot, vec_dict['gtot'].as_numpy()[a:b], 'g', label='gtot')
ax.set_ylabel('conductance (uS)')
ax.legend()
ax.grid(True)
ax.set_title('Single GABA Synapse')

ax = axes[1]
ax.plot(t_plot, vec_dict['Rrp'].as_numpy()[a:b], 'b', label='Rrp')
ax.plot(t_plot, vec_dict['Use'].as_numpy()[a:b], 'r', label='Use')
ax.plot(t_plot, vec_dict['G'].as_numpy()[a:b], 'r', label='G')
ax.grid(True)
ax.legend()

ax = axes[2]
ax.plot(t_plot, vec_dict['Itot'].as_numpy()[a:b], 'b', label='Itot')
ax.grid(True)
ax.legend()

### GABA-A Only

Matching traces from Atherton (2013)
- U1 = 0.1
- tau_rec = 1250.0
- tau_facil = 50.0

In [ ]:
# Calibrate GABAA conductance so we get PSP of desired magnitude

stim_data = {}
istim = None
synapses, stims, netcons = [], [], []

spike_interval = 150.0

for i, seg in enumerate((proximal_seg,)):
    # Insert synapse in selected compartment
    syn = h.GABAsyn(seg)
    
    # Configure incoming spikes
    stim = h.NetStim()
    stim.number = 1e9 # need barrage to trigger signaling cascade
    stim_rate = 33.0 # Hz
    stim.interval = stim_rate**-1*1e3
    stim.noise = 0
    stim.start = 500 + i*spike_interval
    
    nc = h.NetCon(stim, syn)
    nc.delay = 1.0
    
    # Synapse parameters
    # syn.gmax_AMPA = 0.01 # [uS] or set netcon weight in [nS]
    syn.gmax_GABAB = 0.0
    syn.tau_r_GABAA = 2.0
    syn.tau_d_GABAA = 7.0
    
    syn.tau_rec = 1250.0
    syn.tau_facil = 50.0
    syn.U1 = 0.1 # release probability
    
    nc.weight[0] = 7.0 / syn.U1 # conductance in [nS]
    
    # Save refs
    synapses.append(syn)
    stims.append(stim)
    netcons.append(nc)

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'prox': proximal_seg,
    'mid': middle_seg,
    'dist': distal_seg,
    'synGABA': synapses[0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'V_prox': {'var':'v', 'seg':'prox'},
    'gGABAA': {'pointp':'synGABA', 'var':'g_GABAA'},
    'Rrp': {'pointp':'synGABA', 'var':'Rrp'},
    'Use': {'pointp':'synGABA', 'var':'Use'},
}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.set_aCSF(4) # Hoc function defined in Gillies code
h.v_init = -68.0
h.tstop = 2000.0
h.init()
h.run()
# nrnsim.run(h.tstop, h.dt)

# Plot recorded traces
plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.grid(True)

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

### GABA-B Only

In [ ]:
# Calibrate GABAB conductance so we get PSP of desired magnitude

spike_interval = 0.0
synapses, stims, netcons = [], [], []

# NOTE: rise and decay time represent that of first quantity in signaling
#       cascade, NOT those of the conductance or IPSP
conductance_nS = 7.0
tau_rise = 5.0 # 66.0
tau_decay = 10.0 # 100.0

for i, seg in enumerate((proximal_seg,)):
    syn = h.GABAsyn(seg)
    
    stim = h.NetStim()
    stim.number = 8 # need barrage to trigger signaling cascade
    stim_rate = 100.0 # Hz
    stim.interval = stim_rate**-1*1e3
    stim.noise = 0
    stim.start = 500 + i*spike_interval
    
    nc = h.NetCon(stim, syn)
    nc.delay = 1.0
    
    # Synapse parameters
    hill_factor = 0.21
    nc.weight[0] = conductance_nS / hill_factor # conductance in [nS]
    # syn.gmax_AMPA = 0.01 # [uS] or set netcon weight in [nS]
    syn.gmax_GABAA = 0.0
    syn.tau_r_GABAB = tau_rise
    syn.tau_d_GABAB = tau_decay
    syn.U1 = 1.0 # release probability
    
    # Save refs
    synapses.append(syn)
    stims.append(stim)
    netcons.append(nc)

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'prox': proximal_seg,
    'mid': middle_seg,
    'dist': distal_seg,
    'synGABA': synapses[0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'V_prox': {'var':'v', 'seg':'prox'},
    'gGABAA': {'pointp':'synGABA', 'var':'g_GABAB'},
    'Rrp': {'pointp':'synGABA', 'var':'Rrp'},
    'Use': {'pointp':'synGABA', 'var':'Use'},
}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.set_aCSF(4) # Hoc function defined in Gillies code
h.v_init = -68.0
h.tstop = 2000.0
h.init()
# h.run()
nrnsim.run(h.tstop, h.dt)

# Plot recorded traces
plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.grid(True)

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

# ALL Afferents

The Observations Database contains following observations that may help in configuring the distribution of synapses on the STN dendritic tree:

## Experimental rates & contacts

### Afferents & Firing Rate

- each STN neuron receives approximately <span style='color:blue;font-weight:bold'>300 synaptic inputs</span>
    + each with mean conductance of 0.8 nS
    + (this indicates that the author is talking about one synaptic contact of a multi-synaptic connection)


- mean discharge rate of STN neurons 20-25 Hz
    + most STN neurons have a Poisson-like discharge rate with a slight tendency towards bursting

### GPe->STN Synapse Data


- one STN cells receives inputs from maximally <span style='color:blue;font-weight:bold'>2% of GPe cells</span>
    + (`num_cell_gpe = 46.000`)
- <span style='color:blue;font-weight:bold'> Atherton (2013) </span>:
    + "Thus, each GPe neuron may establish ~15.5 synaptic contacts with each STN neuron. As each STN neuron has been estimated to receive an average of 883 synaptic contacts from the GPe (Baufreton et al., 2009), ~57 GPe neurons may provide convergent synaptic input to each STN neuron."
- Unitary GABAAR-mediated GPe-STN conductances are large <span style='color:blue;font-weight:bold'>(~5-15 nS)</span> because each axon forms multiple synaptic inputs with an individual postsynap-tic neuron
    + If we divide this 10 nS by the 0.8 nS mentioned above, we can infer that one afferent axon makes between <span style='color:blue;font-weight:bold'>6-18 contacts</span>. 
    + If we then represent each axon by a single simulated synapse, we can divide the 300 synapses by 6-18 (contacts/synapse) and we get <span style='color:blue;font-weight:bold'>16-50 synapses</span>.
- location / spatal distribution
    - The majority of GP terminals innervate the proximal regions of STN neurons.
    - the strength of pallidal GABAergic synapses remains relatively constant along the dendritic tree of STN neurons.
    - extrasynaptic GABAB receptors are closer to putative glutamatergic than putative GABAergic synapses in the STN.

### CTX->STN Synapse Data

- See parameters in calibration experiment above

### STN->STN Synapse Data

- The number of intra-nucleus end-terminals [resembling boutons] ranged from 5 to 19 (14.4 +/- 2.3, mean +/- std)
    + on average <span style='color:blue;font-weight:bold'>14.4 [boutons/afferent]</span>

### Summary

Number of boutons per afferent population:

|          | TOT | GPe.GABAA   | GPe.GABAB | CTX    | STN          | other      |
|----------|-----|-------------|-----------|--------|--------------|------------|
| boutons  | 300 | ?           | ?         |        |14.4*`n_stn_pre`|            |
| fraction |     |             |           |        |              |            |
| afferents|     |0.2*`num_gpe`|0.2*`num_gpe`|        |              |            |
| strength |     |             |           |        |              |            |
| location |     | proximal    | near GLU  | distal | ?            |            |

- __GPE__: 300 [boutons] * 0.33 [GPE fraction] / 6-18 [boutons/afferent] = 100 / 12 = 8.33 [afferent axons]
- __CTX__: 300 [boutons] * 0.5 [CTX fraction] / 10? [boutons/afferent] = 150 / 10 = 15 [afferent axons]
- __STN__: 300 [boutons] * 0.17 [STN fraction] / 14.4 [boutons/afferent] = 50 / 14.4 = 3.5 [afferent axons]

- __TODO__
    - set realistic cortical rate
    - set fixed number of synapses on STN cells in network model, using same parameters

- __CALIBRATION METHOD__ : tweak until realistic conductances (see plots), balanced currents, and target rate approximated

## Make Synapses

### ALL -> STN (GABA+GLU)

In [ ]:

# Clear old inputs
stim_data = {}
istim = None

# Parameters for afferents
afferent_params = aff_par = {}
h.distance(0, 0.5, sec=soma) # reference for distance measurement

# Set scale factors to 1 divided by number of co-activated contacts resulting in calibrated PSP

aff_par['CTX'] = {}
aff_par['CTX']['nsyn'] = NUM_PRE_CTX = 14 # inputs * contacts, divergent pattern
aff_par['CTX']['segs'] = sample_tree_uniformly(soma, NUM_PRE_CTX, 0.25,
                                              lambda seg: h.distance(1, seg.x, sec=seg.sec) >= 100.0)
aff_par['CTX']['rate_scale'] = SCALE_RATE_CTX = 5
aff_par['CTX']['rate'] = RATE_CTX = (5.0 - 1.0) / 2 * SCALE_RATE_CTX
aff_par['CTX']['gsyn_scale'] = 1.0
aff_par['CTX']['gsyn_base'] = 2e-3


aff_par['GPE'] = {}
aff_par['GPE']['nsyn'] = NUM_PRE_GPE = 8 # inputs * contacts
aff_par['GPE']['segs'] = sample_tree_uniformly(soma, NUM_PRE_GPE, 0.25,
                                              lambda seg: h.distance(1, seg.x, sec=seg.sec) <= 120)
aff_par['GPE']['rate'] = RATE_GPE = 40 # in between pauses
aff_par['GPE']['gsyn_scale'] = 1.0
aff_par['GPE']['gsyn_base'] = 2e-2


make_ctx_afferents(aff_par['CTX']['segs'], rate=aff_par['CTX']['rate'],
                   gsyn_base=aff_par['CTX']['gsyn_base'],
                   scale_gsyn=aff_par['CTX']['gsyn_scale'],
                   noise=1.0, tstart=20,
                   intraburst_rate=100, interburst_interval=500.0, 
                   burst_duration=50.0, burst_noise=1.0)

make_gpe_afferents(aff_par['GPE']['segs'], rate=aff_par['GPE']['rate'],
                   gsyn_base=aff_par['GPE']['gsyn_base'],
                   scale_gsyn=aff_par['GPE']['gsyn_scale'],
                   noise=1.0, tstart=20)
#                    intraburst_rate=60, interburst_interval=250.0, 
#                    burst_duration=2500.0, burst_noise=1.0)

# Check that we have correct number of inputs
for pre in stim_data.keys():
    print("Afferents from {pre} have:\n\t- {nsyn} synapses"
          "\n\t- {nstim} spike generators".format(
              pre=pre, nsyn=len(stim_data[pre]['synapses']),
              nstim=len(stim_data[pre]['netstims'])))

### Weight Scaler

In [ ]:
# Activity-dependent scaling of synaptic weights
from bgcellmodels.mechanisms import plasticity

# Clear old references
hpwa = None
rec_secs = None

hpwa = h.VanRossumHPWA(soma(0.5))
hpwa.scaling = 0
hpwa.sensing = 0
hpwa.activitytau = 1e3 # larger window if variable rate (10-100)
hpwa.activitybeta = 5e-1 # (s^-1 Hz^-1) Scaling strength proportional to activity error (was 4e-5 in article)
hpwa.set_target_rate(22.5) # SETPARAM: target rate

# Add weights to scale
# NOTE: we keep STN calibrated value, so we scale STR inputs
for syn in stim_data['CTX']['synapses']:
    h.setpointer(syn._ref_gmax_AMPA, 'temp_wref', hpwa)
    hpwa.add_wref(1)
    h.setpointer(syn._ref_gmax_NMDA, 'temp_wref', hpwa)
    hpwa.add_wref(1)

# Set up activity sensing
spike_rec = h.NetCon(soma(0.5)._ref_v, hpwa, sec=soma)
spike_rec.threshold = 0.0
spike_rec.delay = 1
spike_rec.weight[0] = 1

# Enable plasticity after delay
cvode = h.CVode()
def enable_sensing():
    hpwa.sensing = 1
    print("Start sensing at time {}".format(h.t))
def enable_scaling():
    hpwa.scaling = 1 # SETPARAM: enable or disable scaling
    print("Start scaling at time {}".format(h.t))
def add_events():
    cvode.event(50.0, enable_sensing)
    cvode.event(2.0*hpwa.activitytau, enable_scaling)
fih = h.FInitializeHandler(add_events)

### Plot Synapse Locations

In [ ]:
# plt.figure()
# ax = plt.subplot(111, projection='3d')
# ax.view_init(90,90) # set azimut and elevation
# ax.set_zlim((-1,1)) # prevent rounding errors
# ax.set_axis_off() # hide x,y,z axes and grid

# # Plot shape of our cell
# h.define_shape(sec=soma) # for cells without 3D morphology
# morphology.shapeplot(h, ax)

# # Plot synapse locations
# for i, syn in enumerate(stim_data['CTX']['synapses']):
#     seg = syn.get_segment()
#     label = 'GLU' if i==0 else None
#     morphology.mark_locations(h, seg.sec, seg.x, label=label, markspec='ob')

# for i, syn in enumerate(stim_data['GPE']['synapses']):
#     seg = syn.get_segment()
#     label = 'GABA' if i==0 else None
#     morphology.mark_locations(h, seg.sec, seg.x, label=label, markspec='or')

# plt.legend()
# plt.show()

### Record & Run

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'soma_spiker': h.NetCon(soma(0.5)._ref_v, None, -10.0, 0, 0),
    'hpwa': hpwa,
#     'prox': proximal_seg,
#     'mid': middle_seg,
#     'dist': distal_seg,
#     'synGABA': synapses[0],
}

trace_specs = {
    't_global': {'var': 't'},
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'AP_soma': {'netcon': 'soma_spiker'},
    # HPWA weight scaling
    'a_HPWA': {'pointp':'hpwa', 'var':'activity'},
    'scale_HPWA': {'pointp':'hpwa', 'var':'scalefactor'},
    # STN Synapse variables
    'gAMPA_ctx': {'pointp':'synSTN0', 'var':'g_AMPA'},
    'gNMDA_ctx': {'pointp':'synSTN0', 'var':'g_NMDA'},
    'iAMPA_ctx': {'pointp':'synSTN0', 'var':'i_AMPA'},
    'iNMDA_ctx': {'pointp':'synSTN0', 'var':'i_NMDA'},
    'itot_ctx':  {'pointp':'synSTN0', 'var':'i'},
    # GPE Synapse variables
    'gGABAA_gpe': {'pointp':'synGPE0', 'var':'g_GABAA'},
    'gGABAB_gpe': {'pointp':'synGPE0', 'var':'g_GABAB'},
    'iGABAA_gpe': {'pointp':'synGPE0', 'var':'i_GABAA'},
    'iGABAB_gpe': {'pointp':'synGPE0', 'var':'i_GABAB'},
    'itot_gpe':  {'pointp':'synGPE0', 'var':'i'},
}

# Record spikes at synapses
for pop in 'CTX', 'GPE':
    for i_syn, nc in enumerate(stim_data[pop]['netcons']):
        con_label = "con{}{}".format(pop, i_syn)
        syn_label = "syn{}{}".format(pop, i_syn)
        rec_secs[con_label] = nc
        rec_secs[syn_label] = nc.syn()
        trace_specs['AP_'+con_label] = {'netcon': con_label}
        trace_specs['i_'+syn_label] = {'pointp': syn_label, 'var':'i'}

# SETPARAM: low sampling interval for slow traces
rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

In [ ]:
# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.set_aCSF(4) # Hoc function defined in Gillies code
h.v_init = -68.0
h.tstop = 5000.0
# h.init()

import time
tstart = time.time()
h.run()

tstop = time.time()
cputime = tstop - tstart
num_segments = sum((sec.nseg for sec in h.allsec()))
print("Simulated {} segments for {} ms in {} ms CPU time".format(
        num_segments, h.tstop, cputime))

## Simulate & Analyze

### Plot Weight Scaling

In [ ]:
# Plot GLU synapse variables
fig, axes = plt.subplots(2, 1, figsize=(10,6), sharex=True)

# Signals
activity_hz = vec_dict['a_HPWA'].as_numpy() * 1e3
scale_vec = vec_dict['scale_HPWA'].as_numpy()
t_rec = np.arange(len(activity_hz)) * rec_dt

# Activity on faster time scale (want to know variability)
bin_width = 100.0
psth_vec = signal.nrn_avg_rate_simple([vec_dict['AP_soma'].as_numpy()], 0, h.tstop, bin_width)
psth_rate = psth_vec.as_numpy()

# Plotted interval
interval = (0.0, h.tstop)
a = int(interval[0] // rec_dt)
b = int(interval[1] // rec_dt)
t_plot = t_rec[a:b]

ax = axes[0]

ax.plot(t_rec, activity_hz, 'b', label='lpf_rate')
ax.plot(np.arange(psth_rate.size)*bin_width, psth_rate, 'g', label='fast_rate')
ax.vlines(vec_dict['AP_soma'].as_numpy(), 
        activity_hz.min(), 0.2*activity_hz.max(),
        color='red', linewidth=1.0)
ax.set_xlim((interval))
ax.set_ylabel('activity (Hz)')
ax.legend()
ax.grid(True)
ax.set_title('HPWA activity sensor')

ax = axes[1]
ax.plot(t_rec, scale_vec, 'g', label='scale')
ax.set_xlim((interval))
ax.set_ylabel('scale_factor')
ax.legend()
ax.grid(True)
ax.set_title('HPWA weight scale factor')


print("Final scale factor is: {}".format(hpwa.scalefactor))
print("Final STN gmax_AMPA is {}".format(stim_data['CTX']['synapses'][1].gmax_AMPA))
print("Final STN gmax_NMDA is {}".format(stim_data['CTX']['synapses'][1].gmax_NMDA))
print("Final GPE gmax_GABAA is {}".format(stim_data['GPE']['synapses'][1].gmax_GABAA))
print("Final GPE gmax_GABAB is {}".format(stim_data['GPE']['synapses'][1].gmax_GABAB))

### Plot Voltage Traces

In [ ]:
# Plot recorded traces
# t_global = vec_dict.pop('t_global').as_numpy() # inconsistent if dt != rec_dt 
interval = (2000, h.tstop)

# Plot somatic voltage traces
v_soma = vec_dict['V_soma'].as_numpy()
t_rec = np.arange(len(v_soma)) * rec_dt
# plt.figure()
# plt.plot(t_rec, v_soma)
# plt.grid(True)
spikes_soma = vec_dict['AP_soma'].as_numpy()
spike_range = (spikes_soma > interval[0]) & (spikes_soma <= interval[1])
soma_rate = len(spikes_soma[spike_range]) / float(interval[1]-interval[0]) * 1e3
analysis.plotTraces(vec_dict, rec_dt, interval, includeTraces=['V_soma'], title="Soma @ {} Hz".format(soma_rate))

### Plot Spike Trains

In [ ]:
# Plot spikes
for pop_label in 'CTX', 'GPE':
    spike_data = analysis.match_traces(
                    vec_dict, 
                    lambda label: label.startswith('AP_con'+pop_label))
    analysis.plotRaster(
                    spike_data, interval,
                    color='b',
                    title='{} {} spiketrains'.format(
                        len(spike_data), pop_label))

### Plot Synaptic Currents

In [ ]:
# Plot synaptic variables
a = int(interval[0] // rec_dt)
b = int(interval[1] // rec_dt)
t_plot = t_rec[a:b]

NTR = { # neurotransmitter receptors
    'CTX': ('AMPA', 'NMDA'),
    'GPE': ('GABAA', 'GABAB'),
}

# Plot GLU synapse variables
for pop_label in 'CTX', 'GPE':
    fig, axes = plt.subplots(2,1)
    
    ntr = NTR[pop_label]
    x_vec0 = vec_dict['g{}_{}'.format(ntr[0], pop_label.lower())].as_numpy()
    x_vec1 = vec_dict['g{}_{}'.format(ntr[1], pop_label.lower())].as_numpy()

    for i in range(len(ntr)):
        # plot conductances
        g_vec = vec_dict['g{}_{}'.format(ntr[i], pop_label.lower())].as_numpy()
        label = 'g{}'.format(ntr[i])
        axes[0].plot(t_plot, g_vec[a:b], 'rgb'[i], label=label)
        
        # plot currents
        i_vec = vec_dict['i{}_{}'.format(ntr[i], pop_label.lower())].as_numpy()
        label = 'i{}'.format(ntr[i])
        axes[1].plot(t_plot, i_vec[a:b], 'rgb'[i], label=label)
    
    # total current
    i_vec = vec_dict['itot_{}'.format(pop_label.lower())].as_numpy()
    axes[1].plot(t_plot, i_vec[a:b], 'b', label='i_tot')
    
    axes[0].set_ylabel('conductance (uS)')
    axes[0].legend()
    axes[0].grid(True)
    axes[0].set_title('Single {} Synapse'.format(pop_label))

    axes[1].set_ylabel('current (nA)')
    axes[1].legend()
    axes[1].grid(True)



# Plot total GABA and GLU current from all synapses
itot_GABA = vec_dict['i_synGPE0'].as_numpy()
for i in range(1, len(stim_data['GPE_synapses'])):
    itot_GABA += vec_dict['i_synGPE{}'.format(i)].as_numpy()

itot_GLU = vec_dict['i_synCTX0'].as_numpy() * -1.0
for i in range(1, len(stim_data['CTX_synapses'])):
    itot_GLU -= vec_dict['i_synCTX{}'.format(i)].as_numpy()

plt.figure()
plt.plot(t_rec, itot_GABA, 'r', label='i_GABA')
plt.plot(t_rec, itot_GLU, 'b', label='i_GLU')
plt.ylabel('current (nA)')
plt.xlabel('time (ms)')
plt.legend()
plt.grid()
plt.show()
# Plot remaining traces
# figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

In [ ]:
# Plot synaptic variables
a = int(interval[0] // rec_dt)
b = int(interval[1] // rec_dt)
t_plot = t_rec[a:b]

# Plot GLU synapse variables
fig, axes = plt.subplots(2, 1, figsize=(10,6))

ax = axes[0]
ax.plot(t_plot, vec_dict['gAMPA'].as_numpy()[a:b], 'b', label='gAMPA')
ax.plot(t_plot, vec_dict['gNMDA'].as_numpy()[a:b], 'r', label='gNMDA')
ax.set_ylabel('conductance (uS)')
ax.legend()
ax.grid(True)
ax.set_title('Single GLU Synapse')

ax = axes[1]
ax.plot(t_plot, vec_dict['iAMPA'].as_numpy()[a:b], 'b', label='iAMPA')
ax.plot(t_plot, vec_dict['iNMDA'].as_numpy()[a:b], 'r', label='iNMDA')
ax.plot(t_plot, vec_dict['iGLU'].as_numpy()[a:b], 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.legend()
ax.grid(True)

# Plot GABA synapse variables
fig, axes = plt.subplots(2, 1, figsize=(10,6))

ax = axes[0]
ax.plot(t_plot, vec_dict['gGABAA'].as_numpy()[a:b], 'b', label='gGABAA')
ax.plot(t_plot, vec_dict['gGABAB'].as_numpy()[a:b], 'r', label='gGABAB')
ax.set_ylabel('conductance (uS)')
ax.legend()
ax.grid(True)
ax.set_title('Single GABA Synapse')

ax = axes[1]
ax.plot(t_plot, vec_dict['iGABAA'].as_numpy()[a:b], 'b', label='iGABAA')
ax.plot(t_plot, vec_dict['iGABAB'].as_numpy()[a:b], 'r', label='iGABAB')
ax.plot(t_plot, vec_dict['iGABA'].as_numpy()[a:b], 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.grid(True)
ax.legend()

# G protein concentration
fig, ax = plt.subplots(1, 1, figsize=(10,4))
ax.plot(t_plot, vec_dict['Gprot'].as_numpy()[a:b], 'b', label='Gprot')
ax.set_ylabel('G protein')
ax.legend()
ax.grid(True)
ax.set_title('G protein concentration')

# Plot total GABA synapse current
itot_GABA = vec_dict['i_synGABA0'].as_numpy()
for i in range(1, len(stim_data['GABA_synapses'])):
    itot_GABA += vec_dict['i_synGABA{}'.format(i)].as_numpy()

itot_GLU = vec_dict['i_synGLU0'].as_numpy() * -1
for i in range(1, len(stim_data['GLU_synapses'])):
    itot_GLU -= vec_dict['i_synGLU{}'.format(i)].as_numpy()

plt.figure(figsize=(10,4))
plt.plot(t_rec, itot_GABA, 'r', label='i_GABA')
plt.plot(t_rec, itot_GLU, 'b', label='i_GLU')
plt.ylabel('current (nA)')
plt.xlabel('time (ms)')
plt.legend()
plt.grid()
plt.show()
plt.suptitle('Total GABA and GLU current')
# Plot remaining traces
# figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),